In [9]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json
from datetime import datetime
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
j = json.load(open("../data/2020-02-24.json"))

In [4]:
len(j)

3391

In [8]:
pprint(j[0])

{'a': {'name': 'Ääsmäe',
       'players': [[False, False, 2, 'Tauno Sau', 1273],
                   [False, False, 4, 'Fredy Maalt', 151],
                   [False, False, 5, 'Ragnar Markus Kuljus', 290],
                   [False, False, 9, 'Ivar Agar', 288],
                   [False, True, 10, 'Andre Simovart', 294],
                   [False, False, 15, 'Targo Trepp', 1932],
                   [False, False, 18, 'Tormi Voog', 472],
                   [False, False, 19, 'Rasmus Ollemaa', 1444],
                   [False, False, 20, 'Rauno Miller', 747],
                   [True, False, 21, 'Rauno Varik', 298],
                   [False, False, 22, 'Urmet Neem', 2099],
                   [False, False, 24, 'Ergo Orumaa', 461],
                   [True, False, 31, 'Siim Stokkeby', 159],
                   [False, False, 34, 'Janar Laaneste', 307],
                   [False, False, 66, 'Sander Stokkeby', 161],
                   [False, False, 78, 'Jaan Rebane', 2105],
              

## Extraction functions

In [35]:
def get_datetime(game):
    return datetime.strptime(game["meta"]["aeg"][:10], "%d.%m.%Y")

def get_season(game):
    dt = get_datetime(game)
    base_year = dt.year
    if dt.month <= 7:
        base_year = dt.year - 1
        
    return str(base_year) + "/" + str(base_year+1)[2:4]

def get_num_people(game):
    return len(game["a"]["players"]) + len(game["b"]["players"])

def get_location(game):
    return game["meta"]["koht"]

def get_league(game):
    return game["meta"]["voistlus"]

def get_features(g):
    return {
        "time": get_datetime(g),
        "season": get_season(g),
        "num_people": get_num_people(g),
        "location": get_location(g),
        "league": get_league(g)
    }


dfl = [get_features(game) for game in j]
df = pd.DataFrame(dfl)

# TODO Fix locations - take only first part of string


df.head()

,league,location,num_people,season,time
0,PAF Saalihokiliiga meistrivõistlused,Kristiine SH,33,2019/20,2020-02-23
1,Naiste esiliiga meistrivõistlused,Mustvee,26,2019/20,2020-02-15
2,Naiste esiliiga meistrivõistlused,Mustvee,29,2019/20,2020-02-15
3,PAF Saalihokiliiga meistrivõistlused,Rapla Sadolin,35,2019/20,2020-02-16
4,PAF Saalihokiliiga meistrivõistlused,Rapla Sadolin,31,2019/20,2020-02-16


## Summary tables

In [28]:
# Count of games by season
df.groupby("season")["season"].aggregate("count")

season
2011/12    483
2012/13    392
2013/14    339
2014/15    370
2015/16    352
2016/17    416
2017/18    417
2018/19    402
2019/20    220
Name: season, dtype: int64

In [29]:
# Count of people by season
df.groupby("season")["num_people"].aggregate("sum")

season
2011/12    13363
2012/13    11119
2013/14     9930
2014/15    10339
2015/16    10321
2016/17    11196
2017/18    10773
2018/19    10815
2019/20     6010
Name: num_people, dtype: int64

In [36]:
# Top locations
df.groupby("location")["location"].aggregate("count").sort_values(ascending=False)

location
Virtuse SH                 732
Kristiine SH               569
EMÜ SH                     427
Õismäe SMH                 268
Tapa SH                    170
Kiili SH                   128
Sparta SH                  105
EMÜ SK                      88
TTÜ SH                      85
Õismäe SH                   76
Kuusalu                     45
Keila TK                    45
Valga SH                    44
Imavere SH                  42
Kristiine                   36
Kuusalu SH                  31
Tapa                        30
Kehra                       29
Rapla                       27
Sadolini SH                 27
Jõgeva                      24
Õismäe                      23
Valga                       18
EMÜ spordihoone             17
Tamsalu SH                  15
Kõrgessaare SH              14
Sadolin SH                  14
Tapa SK                     13
Aruküla                     12
Jõgeva SH                   12
                          ... 
Sadolin Spordihoone          5

## Plots